In [1]:
from scipy import stats
from sklearn.metrics import roc_curve, roc_auc_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

from scipy.stats import ttest_ind
import warnings
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy.stats import norm
import math

warnings.filterwarnings('ignore')
%matplotlib inline

# color palette
palette1 = "#007BFF"
palette2 = "#3ABFC2"
palette3 = "#E1C081"
palette4 = "#E17A57"
palette5 = "#D92D3F"

# load processed results
df = pd.read_csv("data/results/results_processed.csv")

/Users/heart/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/heart/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
df.head()

,PROLIFIC_PID,order,initial_rating,second_rating,knowledge,page_1_time,page_2_time,stimulus_ID,explanation_type,RecordedDate,Duration (in seconds),Comments,trust,NFC,CRT,condition,stimulation,ground_truth,topic,logical_validity
0,6108e03508bd5499cf3c2a71,1,2.3,1.0,2.5,23.300,27.030,1022.0,Honest,2022-12-02 09:13:52,736,Trust AI,3.333333,2.777778,0.000000,flag,trivia,False,The body's ability to self-heal,VALID
1,5f78ce64cb5f144a98a4fa38,1,4.0,5.0,1.0,8.030,9.871,524.0,Honest,2022-12-02 09:15:10,713,NaN,4.500000,3.166667,0.333333,flag,news,True,Biden's policies towards Iran,INVALID
2,6046a861fd56950eb9e314ea,1,4.1,4.1,1.1,16.800,15.842,1038.0,Deceptive,2022-12-02 09:18:10,942,"I really didn’t have a process per se, rather ...",3.500000,3.111111,0.000000,flag+explanation,trivia,False,Scientific names of wolves,VALID
3,5b66c8d4cda8590001dafb0c,1,2.0,4.5,2.0,12.168,14.103,1017.0,Deceptive,2022-12-02 09:18:53,483,I just went with my gut but then trusted the AI,2.833333,2.777778,0.666667,flag+explanation,trivia,False,"The song ""Let it Go"" from Frozen",INVALID
4,5f2c1cc33ea8e3068272d06c,1,1.0,1.0,2.1,5.601,14.100,1019.0,Deceptive,2022-12-02 09:20:48,516,seeing what was actually true,4.000000,3.055556,0.333333,flag+explanation,trivia,False,The history of M&Ms,INVALID


In [3]:
# creating dummy variables aka binary features
df["trivia"] = df.stimulation=="trivia"
df["false_statement"] = (df.ground_truth==True).astype(int)
df["explanation"] = (df.condition=="flag+explanation").astype(int)
df["deceptive_feedback"] = (df.explanation_type=="Deceptive").astype(int)


df["deceptive_feedback_x_explanation"] = df["deceptive_feedback"]*df["explanation"]
df["deceptive_feedback_x_false_statement"] = df["deceptive_feedback"]*df["false_statement"]
df["deceptive_feedback_x_fs_x_explanation"] = df["deceptive_feedback"]*df["explanation"]*df["false_statement"]
df["explanation_x_false_statement"] = df["explanation"]*df["false_statement"]

df["rating_delta"] = df.second_rating - df.initial_rating

In [4]:
# double checking that the dta looks balanced, all good
df.groupby(["trivia","false_statement","explanation","deceptive_feedback"])["PROLIFIC_PID"].nunique()

trivia  false_statement  explanation  deceptive_feedback
False   0                0            0                     288
                                      1                     288
                         1            0                     300
                                      1                     300
        1                0            0                     289
                                      1                     289
                         1            0                     300
                                      1                     300
True    0                0            0                     299
                                      1                     299
                         1            0                     311
                                      1                     311
        1                0            0                     297
                                      1                     299
                         1            0        

In [5]:
x = []
for stimuli in [0,1]:
    for dv in ["initial_rating","second_rating","rating_delta"]:
        i = df[df.trivia==stimuli]
        res = sm.OLS(i[dv], 
                      sm.add_constant(i[[
                          'false_statement',
                          'explanation',
                          "explanation_x_false_statement",
                          'deceptive_feedback',
                          "deceptive_feedback_x_explanation",
                          "CRT",
                      ]]), 
                      M=sm.robust.norms.HuberT()).fit(cov_type='cluster', cov_kwds={'groups': i['PROLIFIC_PID']})
        x.append(res)
stargazer = Stargazer([x[0],x[1],x[2],x[3],x[4],x[5]])
stargazer.covariate_order([
                            'const',
                            'false_statement',
                            'explanation',
                            "explanation_x_false_statement",
                            'deceptive_feedback',
                            "deceptive_feedback_x_explanation",
                            "CRT",
                          ])
stargazer.custom_columns(['N – 1st Rating', 'N – 2nd Rating',"N – Rating Delta",
                         'T – 1st Rating', 'T – 2nd Rating',"T – Rating Delta"],
                         [1,1,1,1,1,1])
stargazer.rename_covariates({
                            'const':'Constant',
                            'false_statement': "False Statement",
                            'explanation':"GPT Explanation",
                            "explanation_x_false_statement": "GPT Explanation * False Statement",
                            'deceptive_feedback': "Deceptive Feedback",
                            "deceptive_feedback_x_explanation": "Deceptive GPT Explanation",
                            })
stargazer.significant_digits(2)
stargazer.show_degrees_of_freedom(False)
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.show_model_numbers(True)
stargazer.add_line('Number of Participants', [
    df[df.trivia==0].PROLIFIC_PID.nunique(),
    df[df.trivia==0].PROLIFIC_PID.nunique(),
    df[df.trivia==0].PROLIFIC_PID.nunique(),
    df[df.trivia==1].PROLIFIC_PID.nunique(),
    df[df.trivia==1].PROLIFIC_PID.nunique(),
    df[df.trivia==1].PROLIFIC_PID.nunique(),
    ])
stargazer.add_line('Number of Stimuli', [
    df[df.trivia==0].topic.nunique(),
    df[df.trivia==0].topic.nunique(),
    df[df.trivia==0].topic.nunique(),
    df[df.trivia==1].topic.nunique(),
    df[df.trivia==1].topic.nunique(),
    df[df.trivia==1].topic.nunique(),
    ])
HTML(stargazer.render_html())
